# Compare AUM of EFTs versus Market Cap of SP500

Also compare individual Stocks Market Capitalization against the SP500.

In [70]:
import pandas as pd
import hvplot.pandas
import sqlalchemy
from dotenv import load_dotenv
from pathlib import Path
import requests
import json
import os
import time
from panel import interact

load_dotenv('data/.env')
AVAPI = os.getenv('Alpha_Vantage_API')

database_connection_string = 'sqlite:///SP500.db'
engine = sqlalchemy.create_engine(database_connection_string)

### Compare Market Capitilization

Market Capitalization of the 3 major SP500 ETFs.

In [71]:
ETF_Data = pd.read_csv(Path("Data/ETFAUM.csv"), index_col='Date', parse_dates=True, infer_datetime_format=True)
ETF_Data = ETF_Data.astype({'SPY': 'float64', 'IVV': 'float64', 'VOO': 'float64', 'SP500': 'float64'})
ETF_Data

,SPY,IVV,VOO,SP500
Date,,,,
2021-12-31,455.22,334.71,279.84,40360.0
2020-12-31,329.02,238.85,176.98,31660.0
2019-12-31,307.39,201.41,130.18,26760.0
2018-12-31,243.39,149.37,89.40,21030.0
2017-12-29,277.54,141.49,83.68,22820.0
2016-12-30,224.82,90.60,56.49,19270.0
2015-12-31,182.04,70.35,40.62,17960.0
2014-12-31,215.91,69.69,27.76,18250.0
2013-12-31,174.85,53.70,14.82,16490.0


In [72]:
ETF_Data['Combined'] = ETF_Data['SPY'] + ETF_Data['IVV'] + ETF_Data['VOO']

In [73]:
ETF_Data['% of SP500'] = ETF_Data['Combined'] / ETF_Data['SP500']

The ETF Data with a combined column that adds up the 3 columns together and another column that displays it as a percentage of the Total SP500 Market Capitalization.

In [74]:
ETF_Data

,SPY,IVV,VOO,SP500,Combined,% of SP500
Date,,,,,,
2021-12-31,455.22,334.71,279.84,40360.0,1069.77,0.026506
2020-12-31,329.02,238.85,176.98,31660.0,744.85,0.023527
2019-12-31,307.39,201.41,130.18,26760.0,638.98,0.023878
2018-12-31,243.39,149.37,89.40,21030.0,482.16,0.022927
2017-12-29,277.54,141.49,83.68,22820.0,502.71,0.022029
2016-12-30,224.82,90.60,56.49,19270.0,371.91,0.019300
2015-12-31,182.04,70.35,40.62,17960.0,293.01,0.016315
2014-12-31,215.91,69.69,27.76,18250.0,313.36,0.017170
2013-12-31,174.85,53.70,14.82,16490.0,243.37,0.014759


In [75]:
ETF_Data_normalized = ETF_Data / ETF_Data.min()
ETF_Data_normalized[{'SP500','Combined'}].hvplot(ylabel = 'normalized Market Cap', title ='S&P 500 Market Capitalization versus the 3 big ETFs AUM(normalized)')

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [76]:
ETF_Data_normalized

,SPY,IVV,VOO,SP500,Combined,% of SP500
Date,,,,,,
2021-12-31,4.771698,12.760579,118.576271,3.543459,8.627873,2.434873
2020-12-31,3.448847,9.105986,74.991525,2.779631,6.007339,2.161200
2019-12-31,3.222117,7.678612,55.161017,2.349429,5.153480,2.193503
2018-12-31,2.551258,5.694624,37.881356,1.846356,3.888701,2.106148
2017-12-29,2.909224,5.394205,35.457627,2.003512,4.054440,2.023667
2016-12-30,2.356604,3.454060,23.936441,1.691835,2.999516,1.772937
2015-12-31,1.908176,2.682043,17.211864,1.576822,2.363174,1.498695
2014-12-31,2.263208,2.656881,11.762712,1.602283,2.527301,1.577313
2013-12-31,1.832809,2.047274,6.279661,1.447761,1.962820,1.355762


Look at Individual stock market cap

In [77]:
SP500_CSSO = pd.read_sql_table('SP500 CSSO', engine)
SP500_Cons_Close = pd.read_sql_table('SP500 Cons Closing', engine)

### -Build Individual Stock Market Caps

Most of this is commented out because it was only necessary once to build the DB.

In [78]:
def BuildMarketCap(stock, csso, close):
    stock_CSSO = csso[{'index',stock}].dropna()
    stock_CSSO['index'] = pd.to_datetime(stock_CSSO['index'])
    stock_close = close[{'index',stock}].dropna()
    stock_close['index'] = pd.to_datetime(stock_close['index'])
    return_df = pd.merge_asof(stock_CSSO, stock_close, on = 'index')
    return_df[f'{stock}'] = return_df[f'{stock}_x'].astype(float) * return_df[f'{stock}_y'].astype(float)
    return_df.drop(columns={f'{stock}_x',f'{stock}_y'}, inplace=True)
    return return_df
    
SP500_list = pd.read_sql_table('SP500_Cons', engine)
SP500_list = SP500_list['0'].tolist()
# SP500_list

In [79]:
# AAPL_MC = BuildMarketCap('AAPL', SP500_CSSO, SP500_Cons_Close)
# merged_MC_df = AAPL_MC

In [80]:
# SP500_index = 1
# target_index = 100
# merged_MC_df

In [81]:
# while SP500_index < target_index:
#     index = SP500_index
#     SP500_index+=1
#     # print(f'Index: {index}, stock: {SP500_list[index]}')
#     try:
#         index_df = BuildMarketCap(SP500_list[index], SP500_CSSO, SP500_Cons_Close)
#     except:
#         print(f'Skipped: {SP500_list[index]}')
#         continue
#     merged_MC_df = pd.merge(merged_MC_df, index_df[SP500_list[index]], how='outer', left_index=True, right_index=True;)

In [82]:
# Careful Writes to DB
# merged_MC_df.to_sql('SP500_MC', engine)

In [83]:
SP500_MC = pd.read_sql_table('SP500_MC', engine)

In [84]:
SP500_MC.set_index('index', inplace=True)

In [85]:
SP500_MC.drop(columns={'level_0'}, inplace=True)

In [86]:
SP500_MC.head()

,AAPL,GOOG,MSFT,AMZN,FB,JPM,JNJ,XOM,BAC,WMT,...,MDLZ,COP,GD,CL,GM,ANTM,EOG,AMT,NOC,SYK
index,,,,,,,,,,,,,,,,,,,,,
2017-03-31,7.478674e+11,5.741451e+11,5.086368e+11,4.237661e+11,4.106666e+11,3.072959e+11,3.127375e+11,3.743985e+11,2.221630e+11,2.043579e+11,...,6.586932e+10,5.416192e+10,5.221558e+10,5.779065e+10,5.304000e+10,3.791896e+10,5.735777e+10,4.513619e+10,4.154491e+10,4.492875e+10
2017-06-30,7.445520e+11,6.296372e+11,5.313124e+11,4.646400e+11,4.378420e+11,3.163733e+11,3.371070e+11,3.474764e+11,2.352911e+11,2.281713e+11,...,6.560561e+10,6.249164e+10,5.649153e+10,6.475119e+10,5.239500e+10,4.383809e+10,5.598736e+10,5.180614e+10,4.470618e+10,4.923710e+10
2017-09-30,7.900501e+11,6.663800e+11,5.750628e+11,4.633707e+11,4.962065e+11,3.266727e+11,3.561115e+11,3.420530e+11,2.396431e+11,2.406099e+11,...,6.127462e+10,6.200237e+10,5.938840e+10,6.551609e+10,5.653200e+10,4.949624e+10,5.199822e+10,5.678843e+10,5.008563e+10,5.190372e+10
2017-12-31,8.599678e+11,7.270209e+11,6.590857e+11,5.660235e+11,5.134986e+11,3.375992e+11,3.490248e+11,3.473493e+11,2.649924e+11,2.602711e+11,...,6.407160e+10,5.437109e+10,6.129265e+10,6.415899e+10,5.738600e+10,4.887601e+10,5.560451e+10,5.866903e+10,5.342862e+10,5.311548e+10
2018-03-31,8.293839e+11,7.170373e+11,7.018663e+11,7.019599e+11,4.643497e+11,3.663016e+11,3.751203e+11,3.545500e+11,3.036812e+11,3.151096e+11,...,6.213597e+10,6.068332e+10,6.040341e+10,6.599620e+10,5.087600e+10,5.762167e+10,6.209077e+10,6.117975e+10,6.088033e+10,5.797210e+10


In [87]:
SP500_MC_cleaned = SP500_MC.dropna(axis=1)

In [88]:
# SP500_MC_cleaned.drop(columns={'COP'},inplace=True)

### -SP500 Constituent Stock Market Capitalization

In [89]:
def plot_MC(ticker):
    return SP500_MC_cleaned[ticker].hvplot()
choices=list(SP500_MC_cleaned.columns)
interact(plot_MC, ticker=choices)

Column
    [0] Column
        [0] Select(name='ticker', options=['AAPL', 'GOOG', ...], value='AAPL')
    [1] Row
        [0] HoloViews(Curve, name='interactive09027')

Get SP500 Market Cap by adding all the stocks up for the same dates

In [90]:
SP500_Market_Cap = SP500_MC_cleaned.sum(axis=1)

In [91]:
SP500_Market_Cap_Normalized = SP500_Market_Cap / SP500_Market_Cap.min()
SP500_Market_Cap_Normalized = pd.DataFrame.from_dict(SP500_Market_Cap_Normalized)
SP500_Market_Cap_Normalized.rename(columns={0: 'SP500_Calc'}, inplace=True)
SP500_Market_Cap_Normalized

,SP500_Calc
index,
2017-03-31,1.000000
2017-06-30,1.031548
2017-09-30,1.071965
2017-12-31,1.147064
2018-03-31,1.185066
2018-06-30,1.187181
2018-09-30,1.233464
2018-12-31,1.183573
2019-03-31,1.227017


In [92]:
SP500_MC_normalized = SP500_MC_cleaned / SP500_MC_cleaned.min()


SP500_MC_normalized.head()

,AAPL,GOOG,MSFT,AMZN,FB,JPM,JNJ,XOM,BAC,WMT,...,MDLZ,COP,GD,CL,GM,ANTM,EOG,AMT,NOC,SYK
index,,,,,,,,,,,,,,,,,,,,,
2017-03-31,1.004453,1.000000,1.000000,1.000000,1.089637,1.102546,1.000000,2.579439,1.206224,1.000000,...,1.129375,-0.418762,1.367596,1.125195,1.823182,1.000000,2.760458,1.000000,1.000000,1.000000
2017-06-30,1.000000,1.096652,1.044581,1.096454,1.161743,1.135115,1.077923,2.393958,1.277502,1.116528,...,1.124853,-0.483165,1.479589,1.260718,1.801011,1.156099,2.694504,1.147774,1.076093,1.095893
2017-09-30,1.061108,1.160647,1.130596,1.093459,1.316604,1.172068,1.138691,2.356593,1.301131,1.177395,...,1.050595,-0.479382,1.555462,1.275611,1.943215,1.305316,2.502519,1.258157,1.205578,1.155245
2017-12-31,1.155014,1.266267,1.295789,1.335698,1.362485,1.211271,1.116031,2.393082,1.438764,1.273604,...,1.098552,-0.420379,1.605336,1.249188,1.972570,1.288960,2.676078,1.299822,1.286045,1.182216
2018-03-31,1.113937,1.248878,1.379897,1.656480,1.232077,1.314252,1.199473,2.442692,1.648823,1.541950,...,1.065364,-0.469183,1.582046,1.284958,1.748797,1.519600,2.988243,1.355448,1.465410,1.290312


In [93]:
ETF_Data.head()

,SPY,IVV,VOO,SP500,Combined,% of SP500
Date,,,,,,
2021-12-31,455.22,334.71,279.84,40360.0,1069.77,0.026506
2020-12-31,329.02,238.85,176.98,31660.0,744.85,0.023527
2019-12-31,307.39,201.41,130.18,26760.0,638.98,0.023878
2018-12-31,243.39,149.37,89.40,21030.0,482.16,0.022927
2017-12-29,277.54,141.49,83.68,22820.0,502.71,0.022029


In [94]:
Merged_MC = pd.merge(ETF_Data_normalized[['SP500', 'Combined']], SP500_MC_normalized, how ='outer', left_index=True, right_index=True)

In [95]:
Merged_MC = pd.merge(SP500_Market_Cap_Normalized, Merged_MC, how ='outer', left_index=True, right_index=True)
Merged_MC.drop(columns='COP', inplace = True)

In [96]:
Merged_MC

,SP500_Calc,SP500,Combined,AAPL,GOOG,MSFT,AMZN,FB,JPM,JNJ,...,DHR,MDLZ,GD,CL,GM,ANTM,EOG,AMT,NOC,SYK
2011-12-30,NaN,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-12-31,NaN,1.118525,1.325591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-12-31,NaN,1.447761,1.962820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-12-31,NaN,1.602283,2.527301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-31,NaN,1.576822,2.363174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-12-30,NaN,1.691835,2.999516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-31,1.000000,NaN,NaN,1.004453,1.000000,1.000000,1.000000,1.089637,1.102546,1.000000,...,1.000000,1.129375,1.367596,1.125195,1.823182,1.000000,2.760458,1.000000,1.000000,1.000000
2017-06-30,1.031548,NaN,NaN,1.000000,1.096652,1.044581,1.096454,1.161743,1.135115,1.077923,...,1.101808,1.124853,1.479589,1.260718,1.801011,1.156099,2.694504,1.147774,1.076093,1.095893
2017-09-30,1.071965,NaN,NaN,1.061108,1.160647,1.130596,1.093459,1.316604,1.172068,1.138691,...,1.088063,1.050595,1.555462,1.275611,1.943215,1.305316,2.502519,1.258157,1.205578,1.155245
2017-12-29,NaN,2.003512,4.054440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
Merged_MC.loc['2017-12-31','SP500'] = Merged_MC.loc['2017-12-29','SP500']
Merged_MC.loc['2017-12-31','Combined'] = Merged_MC.loc['2017-12-29','Combined']

In [98]:
# dropthe dates i don't want
Merged_MC.drop(['2017-12-29','2016-12-30', '2015-12-31', '2014-12-31','2013-12-31','2012-12-31', '2011-12-30'], inplace=True)

In [103]:
def plot_MC(ticker):
    return Merged_MC.hvplot(y=[ticker, 'SP500_Calc',], title = 'S&P500 Market Capitalization versus Individual Stock', ylabel='Normalized Market Capitalization', xlabel='Date')
choices=list(Merged_MC.columns)
interact(plot_MC, ticker=choices)

Column
    [0] Column
        [0] Select(name='ticker', options=['SP500_Calc', ...], value='SP500_Calc')
    [1] Row
        [0] HoloViews(NdOverlay, name='interactive11327')